# Gibbs State Preparation

<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## Overview

This tutorial will show how to train a quantum neural network (QNN) through Paddle Quantum to prepare a quantum Gibbs state.

### Background

The frontiers of quantum computing include quantum machine learning and quantum optimization. In these two areas, the preparation of specific quantum states is a fundamental problem. In particular, the preparation of Gibbs state is a necessary step to realize many quantum algorithms and is widely used in:
- Learning of restricted Boltzmann machines in quantum machine learning [1]
- Solving optimization problems such as convex optimization and positive semidefinite programming [2]
- Combinatorial optimization problem [3]

The Gibbs state is defined as follows: Given the Hamiltonian $H$ of an $n$-qubit system (generally this is a Hermitian matrix of $2^n\times2^n$), the Gibbs state at temperature $T$ is
$$
\rho_G = \frac{{{e^{-\beta H}}}}{{\text{tr}({e^{-\beta H}})}},
\tag{1}
$$
where ${e^{-\beta H}}$ is the matrix exponential of matrix $-\beta H$. $\beta = \frac{1}{{kT}}$ is the inverse temperature parameter of the system, where $T $ Is the temperature parameter and $k$ is Boltzmann's constant (in this tutorial, we take $k = 1$).

## Paddle Quantum Implementation

First, let us import the necessary libraries and packages through the following code.

In [8]:
import scipy
import paddle
from numpy import trace as np_trace
import paddle_quantum
from paddle_quantum.ansatz import Circuit
from paddle_quantum.state import zero_state, State
from paddle_quantum.qinfo import state_fidelity, partial_trace, pauli_str_to_matrix

As a hands-on example, here we consider a 3-qubit Hamiltonian and its Gibbs state:

$$
H = -Z \otimes Z \otimes I - I \otimes Z \otimes Z - Z \otimes I \otimes Z, \quad I=\left [
\begin{matrix}
1 & 0  \\
0 & 1  \\
\end{matrix} 
\right ], \quad 
Z=\left [
\begin{matrix}
1 & 0  \\
0 & -1  \\
\end{matrix} 
\right ].
\tag{2}
$$

In this example, we set the inverse temperature parameter to $\beta = 1.5$. Besides, to test the final results, we have generated the ideal Gibbs state $\rho_G$ in advance according to the definition.

In [9]:
N = 4        # The width of the QNN
N_SYS_B = 3  # The number of qubits of subsystem B used to generate the Gibbs state
SEED = 16    # Fixed random seed
beta = 1.5   # Set the inverse temperature parameter beta

In [10]:
# Generate a specific Hamiltonian represented by a Pauli string
H = [[-1.0,'z0,z1'], [-1.0,'z1,z2'], [-1.0,'z0,z2']]

# Generate matrix information of Hamiltonian
hamiltonian = pauli_str_to_matrix(H, N_SYS_B).numpy()

# Generate the ideal target Gibbs state rho
rho_G = scipy.linalg.expm(-1 * beta * hamiltonian) / np_trace(scipy.linalg.expm(-1 * beta * hamiltonian))

# Set to the data type supported by Paddle Quantum
hamiltonian = hamiltonian.astype("complex64")
rho_G = paddle.to_tensor(rho_G, dtype="complex64")

### Building a quantum neural network

- In this example, we will prepare the Gibbs state by training the QNN (also can be understood as a parameterized quantum circuit). Here, we provide a simple 4-qubit quantum circuit as follows:

  ![Ugibbs.jpg](https://release-data.cdn.bcebos.com/PIC%2FUgibbs.jpg)

- We need to preset some circuit parameters. For example, the circuit has 4 qubits. The first qubit is the ancillary system, and the 2-4th qubits are the subsystems used to generate the Gibbs state.
- Initialize the variable ${\bf{\theta }}$ that represents the vector of parameters in our QNN.
         

Next, we use Paddle Quantum's `Circuit` class and the built-in `real_entangled_layer` circuit template to build a QNN based on the circuit design in the above figure.

In [11]:
def U_theta(num_qubits: int, depth: int) -> Circuit:
    """
    Quantum Neural Network
    """
    # Initialize the QNN according to the number of qubits
    cir = Circuit(num_qubits)
    
    # Built-in {R_y + CNOT} circuit template
    cir.real_entangled_layer(depth=depth)
    
    # The QNN acts on a given initial state
    cir.ry()
    
    return cir

### Configuring the training model: loss function

- Now that we have the data and QNN architecture, we also need to define appropriate training parameters, models, and loss functions to achieve our goals.

- Specifically, we refer to the method in the paper [4]. The core idea is to use the Gibbs state to achieve the minimum free energy.

- By applying the QNN $U(\theta)$ on the initial state, we can get the output state $\left| {\psi \left( {\bf{\theta }} \right)} \right\rangle $. Its state in the 2-4th qubits is recorded as $\rho_B(\theta)$.

- Set the loss function in the training model. In Gibbs state learning, we use the truncation of the von Neumann entropy function to estimate the free energy, and the corresponding loss function, as in reference [4], can be set as $loss = {L_1} + {L_2} + {L_3} $, where

$$
{L_1}= \text{tr}(H\rho_B), \quad {L_2} = 2{\beta^{-1}}{\text{tr}}(\rho_B^2), \quad L_3 = - {\beta ^{ - 1}}\big(\text{tr}(\rho_B^3) + 3\big)/2.
\tag{3}
$$

In [12]:
 # define loss function
def loss_func(cir: Circuit, Hamiltonian: paddle.Tensor, N_SYS_B: int) -> paddle.Tensor:
    # Apply QNN
    rho_AB = cir(zero_state(N))
    
    # Calculate partial trace to obtain the quantum state rho_B of subsystem B
    rho_B = partial_trace(rho_AB, 2 ** (N - N_SYS_B), 2 ** (N_SYS_B), 1)
    
    # Calculate the three parts of the loss function
    rho_B_squre = rho_B @ rho_B
    loss1 = paddle.real(paddle.trace(rho_B @ Hamiltonian))
    loss2 = paddle.real(paddle.trace(rho_B_squre)) * 2 / beta
    loss3 = -(paddle.real(paddle.trace(rho_B_squre @ rho_B)) + 3) / (2 * beta)
    
    # 最终的损失函数
    loss = loss1 + loss2 + loss3  
    
    return loss, rho_B

### Configure training model: model parameters

Before training the QNN, we also need to set some training hyperparameters, mainly the learning rate (LR), the number of iterations (ITR), and the depth (D) of the QNN. Here we set the learning rate to 0.5 and the number of iterations to 50. Readers may wish to adjust by themselves to explore the influence of hyperparameter adjustment on the training effect.

In [13]:
ITR = 50 # Set the total number of iterations of training
LR = 0.5 # Set the learning rate
D = 1 # Set the depth of the QNN

### Training

- After all the training model parameters are set, we convert the data into Tensor in PaddlePaddle and then train the QNN.
- During training, we use [Adam Optimizer](https://www.paddlepaddle.org.cn/documentation/docs/en/api/paddle/optimizer/adam/Adam_en.html). Other optimizers are also provided in PaddlePaddle.
- We output the results of the training process in turn.
- In particular, we sequentially output the fidelity of the quantum state $\rho_B(\theta)$ and Gibbs state $\rho_G$ we learned. The higher the fidelity, the closer the QNN output state is to Gibbs state.

In [14]:
paddle.seed(SEED)
    
# Set calculation mode as density matrix    
paddle_quantum.set_backend(backend='density_matrix')   
    
# Convert Numpy array to Tensor supported in PaddlePaddle
H = paddle.to_tensor(hamiltonian)

# Determine the parameter dimension of the network
circuit = U_theta(N, D)

# Generally speaking, we use Adam optimizer to get relatively good convergence
# Of course, it can be changed to SGD or RMS prop.
opt = paddle.optimizer.Adam(learning_rate=LR, parameters=circuit.parameters())

# Optimization loops
for itr in range(1, ITR + 1):
        
    # Run forward propagation to calculate the loss function and return the generated quantum state rho_B
    loss, rho_B = loss_func(circuit, H, N_SYS_B)
        
    # Run back propagation to minimize the loss function
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

    # Convert to Numpy array to calculate the fidelity of the quantum state F(rho_B, rho_G)
    fid = state_fidelity(State(rho_B), State(rho_G))
        
    # Print training results
    if itr % 10 == 0:
        print('iter:', itr, 'loss:', '%.4f' % loss.numpy(), 'fid:', '%.4f' % fid.numpy())
    if itr == ITR:
        print("\nThe trained circuit:  ")
        print(circuit)

iter: 10 loss: -3.1084 fid: 0.9241
iter: 20 loss: -3.3375 fid: 0.9799
iter: 30 loss: -3.3692 fid: 0.9897
iter: 40 loss: -3.3990 fid: 0.9929
iter: 50 loss: -3.4133 fid: 0.9959

The trained circuit:  
--Ry(6.290)----*--------------x----Ry(0.260)--
               |              |               
--Ry(4.745)----x----*---------|----Ry(6.249)--
                    |         |               
--Ry(-0.01)---------x----*----|----Ry(-0.05)--
                         |    |               
--Ry(0.017)--------------x----*----Ry(6.310)--
                                              


## Conclusion

According to the results obtained from the above training, after about 50 iterations, we can achieve a high-precision Gibbs state with a fidelity higher than 99.5% and complete the preparation of the Gibbs state efficiently and accurately. We can output the QNN's learned parameters and its output state through the print function.

_______

## References

[1] Kieferová, M. & Wiebe, N. Tomography and generative training with quantum Boltzmann machines. [Phys. Rev. A 96, 062327 (2017).](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.96.062327)

[2] Brandao, F. G. S. L. & Svore, K. M. Quantum Speed-Ups for Solving Semidefinite Programs. [in 2017 IEEE 58th Annual Symposium on Foundations of Computer Science (FOCS) 415–426 (IEEE, 2017). ](https://ieeexplore.ieee.org/abstract/document/8104077)

[3] Somma, R. D., Boixo, S., Barnum, H. & Knill, E. Quantum Simulations of Classical Annealing Processes. [Phys. Rev. Lett. 101, 130504 (2008).](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.101.130504)

[4] Wang, Y., Li, G. & Wang, X. Variational quantum Gibbs state preparation with a truncated Taylor series. [Phys. Rev. A 16, 054035 (2021).](https://journals.aps.org/prapplied/abstract/10.1103/PhysRevApplied.16.054035)